<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/swiss_class_reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
import numpy as np
from datasets import load_metric
from transformers import set_seed
set_seed(42)
dataset = load_dataset("swiss_judgment_prediction", "de")
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=2)

In [ ]:
#metric with huggingface
#def compute_metrics(eval_pred):
#    load_f1 = load_metric("f1") # use evaluate.load("f1")
#    logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
#    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
#    return { "f1": f1}
#
#from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
#repo_name = "Direct_v1"

In [11]:
  # sklearn implementation 
from sklearn.metrics import f1_score
from transformers import EvalPrediction 
def compute_metrics(p: EvalPrediction):
        logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = np.argmax(logits, axis=1)
        macro_f1 = f1_score(y_true=p.label_ids, y_pred=preds, average='macro', zero_division=0)
        micro_f1 = f1_score(y_true=p.label_ids, y_pred=preds, average='micro', zero_division=0)
        return {'macro-f1': macro_f1, 'micro-f1': micro_f1}

In [5]:
training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=3e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy="epoch",
    evaluation_strategy ='epoch',
    push_to_hub=False,
    save_total_limit = 5,
    metric_for_best_model="f1",    
    greater_is_better = 1
)
# reference hyperparameters 
#        --metric_for_best_model micro-f1 \
#        --greater_is_better 1 \
#        --evaluation_strategy  epoch \
#        --save_strategy epoch \
#        --save_total_limit  5 \
#        --num_train_epochs  20 \
#        --learning_rate 3e-5 \
#        --per_device_train_batch_size 6 \
#        --per_device_eval_batch_size  6 \
#        --fp16 \
#        --fp16_full_eval \
#        --max_seq_length  4096

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,)

trainer.train()